# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
# Load the csv exported in Part 1 WeatherPy to a dataframe
cities_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Provideniya,64.38,-173.30,49.80,63,6,5.68,RU,1595124726
1,Yeppoon,-23.13,150.73,73.40,38,0,6.93,AU,1595124797
2,Puerto Ayora,-0.74,-90.35,70.00,92,79,3.00,EC,1595124797
3,Kapaa,22.08,-159.32,84.99,83,75,11.41,US,1595124797
4,Brus Laguna,15.75,-84.48,77.41,91,0,3.24,HN,1595124797
...,...,...,...,...,...,...,...,...,...
586,Santa Cruz Cabrália,-16.28,-39.02,68.00,94,75,3.36,BR,1595124894
587,Abu Dhabi,24.47,54.37,96.80,41,0,5.82,AE,1595124891
588,Wajid,3.81,43.25,72.57,70,77,22.39,SO,1595124894
589,Chilca,-12.52,-76.74,60.80,77,40,9.17,PE,1595124894


### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_df[['Lat', 'Lng']]

# Use humidity as the weight
humidity = cities_df['Humidity']

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity = 80,
                                point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [12]:
# Find cities in Australia that meet ideal weather conditions
ideal_cities_df = cities_df[
    (cities_df['Max Temp'] >= 70) &
    (cities_df['Max Temp'] <= 85) &
    (cities_df['Cloudiness'] <= 40) &
    (cities_df['Country'] == 'AU')]

# View dataframe
ideal_cities_df

# No missing values in dataframe
# No need to drop any rows

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Yeppoon,-23.13,150.73,73.4,38,0,6.93,AU,1595124797
303,Kununurra,-15.77,128.73,84.2,15,40,11.41,AU,1595124858
316,Katherine,-14.47,132.27,82.4,32,0,6.93,AU,1595124860
381,Mount Isa,-20.73,139.50,78.8,11,0,6.93,AU,1595124868
525,Hervey Bay,-25.30,152.85,72.0,56,0,5.99,AU,1595124887


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Store needed variables in a dataframe
hotel_df = ideal_cities_df.loc[:, ['City','Country', 'Lat', 'Lng', 'Humidity']]
hotel_df

# Set up additional column to hold information
hotel_df['Hotel Name'] = ""

# Reset and view the dataframe
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Yeppoon,AU,-23.13,150.73,38,
1,Kununurra,AU,-15.77,128.73,15,
2,Katherine,AU,-14.47,132.27,32,
3,Mount Isa,AU,-20.73,139.50,11,
4,Hervey Bay,AU,-25.30,152.85,56,


In [14]:
# Find the first hotel for each city located within 5000 m
# Set url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # Set the coordinates to be iterated
    lat=row['Lat']
    lng=row['Lng']
    coord=f'{lat}, {lng}'
    
    # Set parameters
    params = {
    "location": coord,  
    "radius": 5000,
    "type": "lodging",
    "key": g_key,}
      
    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    # Store results to dataframe
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

In [15]:
# View updated dataframe
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Yeppoon,AU,-23.13,150.73,38,Villa Mar Colina
1,Kununurra,AU,-15.77,128.73,15,Hotel Kununurra
2,Katherine,AU,-14.47,132.27,32,Pine Tree Motel
3,Mount Isa,AU,-20.73,139.50,11,ibis Styles Mt Isa Verona
4,Hervey Bay,AU,-25.30,152.85,56,Shelly Bay Resort


In [20]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
# Store ideal cities' latitude and longitude in locations
city_locations = hotel_df[['Lat', 'Lng']]

# Use ideal cities' humidity as the weight
city_humidity = hotel_df['Humidity']

# Plot Heatmap
fig2 = gmaps.figure()

# Create & add heat layer
heat_layer = gmaps.heatmap_layer(city_locations, weights=city_humidity, 
                                 dissipating=False, max_intensity = 80,
                                point_radius = 1)
fig2.add_layer(heat_layer)

# Create & add marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig2.add_layer(marker_layer)

# Display figure
fig2

Figure(layout=FigureLayout(height='420px'))